In [2]:
from pathlib import Path
import numpy as np
from PIL import Image
import svgwrite

# Create output directory if it doesn't exist
Path("./svg").mkdir(exist_ok=True)

def image_to_svg(input_path, output_path, threshold=128):
    """Convert a PNG image to SVG using edge detection and path tracing."""
    img = Image.open(input_path).convert('L')
    img_array = np.array(img)
    
    dwg = svgwrite.Drawing(output_path, size=(img.width, img.height))
    dwg.add(dwg.rect(insert=(0, 0), size=(img.width, img.height), fill='white'))
    
    for y in range(img.height):
        for x in range(img.width):
            if img_array[y, x] < threshold:
                dwg.add(dwg.rect(insert=(x, y), size=(1, 1), fill='black'))
    
    dwg.save()

# Process all PNG files
for png_file in Path("./png").glob("*.png"):
    svg_file = Path("./svg") / f"{png_file.stem}.svg"
    print(f"Converting {png_file} to {svg_file}")
    image_to_svg(png_file, svg_file)

print("Conversion complete!")

Converting png/2.png to svg/2.svg
Converting png/3.png to svg/3.svg
Converting png/1.png to svg/1.svg
Conversion complete!
